In [110]:
from google.cloud import bigquery
import json
import pathlib

In [3]:
client = bigquery.Client()

project_id = 'dataanalytics-347914'

In [136]:
def get_table_schema(table: str) -> dict:

    path = pathlib.Path('.')
    path = path.cwd().parents[1]

    schema = json.load(open(path/'data/retail_db/schemas.json'))[table]
    schema = sorted(schema, key=lambda col: col['column_position'])

    for i in schema:
        if i['data_type'] == 'float':
            i['data_type'] = 'DECIMAL'
        else:
            i['data_type']  = i['data_type'].upper()

    # table_schema = dict([(i['column_name'], i['data_type'].upper()) for i in schema])

    return schema


#### order_items

In [4]:
!gsutil ls gs://airetail_mld/retail_db/order_items

gs://airetail_mld/retail_db/order_items/part-00000


In [154]:
for table in ['orders', 'order_items', 'products']:
    table_id = f"{project_id}.retail.{table}"
    
    # get schema from json
    schema_dict = get_table_schema(table)
    bigquerySchema = [] 
    for col in schema_dict:
        bigquerySchema.append(bigquery.SchemaField(col['column_name'], col['data_type']))

    # define schema
    job_config = bigquery.LoadJobConfig(
        schema=bigquerySchema
    )

    uri = f"gs://airetail_mld/retail_db/{table}/part-00000"

    # create job
    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    # execute job
    load_job.result()  # Wait for the job to complete.

In [155]:
for table in ['orders', 'order_items', 'products']:
    
    table_id = f"{project_id}.retail.{table}"

    # gets reference to table object, but it only has its metadata
    table = client.get_table(table_id)
    print("Loaded {} rows to table {}".format(table.num_rows, table_id))

Loaded 68883 rows to table dataanalytics-347914.retail.orders
Loaded 172198 rows to table dataanalytics-347914.retail.order_items
Loaded 1345 rows to table dataanalytics-347914.retail.products
